## Imports

In [1]:
import pandas as pd
from datetime import datetime as dt
from pathlib import Path
from hbt import utils_hbt as uh

## Settings

In [2]:
path_raindata = Path(r"Q:\Projekte\10000-\10208.10 Tangente Zug"
    r"\05 Berechnungen Grundlagen\Regendaten\Cham Niederschlag"
    r"\CHZ_Niederschlagsumme.csv")

save_dir = Path(r"Q:\Projekte\10000-\10208.10 Tangente Zug"
    r"\05 Berechnungen Grundlagen\Probenehmer")

dt_start = "21.04.2022 00:00:00"
dt_end = "21.09.2022 13:14:56"

periods_inflow = [
    ('21.04.2022 11:15', '25.04.2022 03:16'),
    ('25.04.2022 03:16', '26.04.2022 10:53'),
    ('26.04.2022 11:53', '06.05.2022 02:31'),
    ('06.05.2022 02:31', '25.05.2022 12:10'),
    ('01.06.2022 14:47', '07.06.2022 08:11'),
    ('07.06.2022 08:11', '22.06.2022 21:57'),
    ('22.06.2022 21:57', '01.07.2022 00:03'),
    ('01.07.2022 00:03', '04.07.2022 10:10'),
    ('04.07.2022 17:01', '29.07.2022 17:57'),
    ('29.07.2022 17:57', '19.08.2022 05:38'),
    ('19.08.2022 05:38', '20.08.2022 19:55'),
]

periods_outflow = [
    ('21.04.2022 12:26', '26.04.2022 21:44'),
    ('26.04.2022 21:44', '05.05.2022 14:21'),
    ('05.05.2022 14:21', '01.06.2022 14:19'),
    ('01.06.2022 17:44', '09.06.2022 18:18'),
    ('09.06.2022 18:18', '04.07.2022 10:23'),
    ('04.07.2022 10:23', '26.08.2022 08:30'),
]

## Read data

In [3]:
discharge_in = uh.hbtdb_get_data(917, dt_from=dt_start, dt_to=dt_end)
discharge_in[discharge_in < 0] = 0 # 73 values were negative
discharge_out = uh.hbtdb_get_data(918, dt_from=dt_start, dt_to=dt_end)
ts_rain = uh.read_raindata_meteoschweiz(path_raindata)

## Analysis

### Flow analysis

In [13]:
# Inflow 01.01.2022 - 31.08.2022
dt_format = '%d.%m.%Y %H:%M'
dt1 = dt.strptime('01.01.2022 00:00', dt_format)
dt2 = dt.strptime('01.09.2022 00:00', dt_format)

mask = (discharge_in.index >= dt1) & (discharge_in.index < dt2)
_discharge_in = discharge_in.loc[mask]
_discharge_in



,wert
datum,
2022-04-21 10:41:51,0.0
2022-04-21 10:42:21,0.0
2022-04-21 10:42:51,0.0
2022-04-21 10:43:21,0.0
2022-04-21 10:43:51,0.0
...,...
2022-08-31 23:55:50,0.0
2022-08-31 23:56:50,0.0
2022-08-31 23:57:50,0.0


In [ ]:
dt_format = '%d.%m.%Y %H:%M'
inflows = []
outflows = []
rain = []
for period in periods_inflow:
    dt_1 = dt.strptime(period[0], dt_format)
    dt_2 = dt.strptime(period[1], dt_format)

    # Rain
    tot_rain = uh.ts_sum_between(ts_rain, dt_1, dt_2)
    rain.append(tot_rain)

    # Inflow
    tot_inflow = uh.ts_sum_between(discharge_in, dt_1, dt_2) * 60 / 1e3
    inflows.append(tot_inflow)

    # Outflow
    tot_outflow = uh.ts_sum_between(discharge_out, dt_1, dt_2) * 60 / 1e3
    outflows.append(tot_outflow)

df_zu = pd.DataFrame(columns=['rain', 'inflow', 'outflow'])

df_zu['inflow'] = inflows
df_zu['outflow'] = outflows
df_zu['rain'] = rain

save_path = save_dir / 'zulauf.csv'
df_zu.to_csv(save_path, sep=';')
#-------------------------------------------------------------------------------

inflows = []
outflows = []
rain = []
for period in periods_outflow:
    dt_1 = dt.strptime(period[0], dt_format)
    dt_2 = dt.strptime(period[1], dt_format)

    # Rain
    tot_rain = uh.ts_sum_between(ts_rain, dt_1, dt_2)
    rain.append(tot_rain)

    # Inflow
    tot_inflow = uh.ts_sum_between(discharge_in, dt_1, dt_2) * 60 / 1e3
    inflows.append(tot_inflow)

    # Outflow
    tot_outflow = uh.ts_sum_between(discharge_out, dt_1, dt_2) * 60 / 1e3
    outflows.append(tot_outflow)

df_ab = pd.DataFrame(columns=['rain', 'inflow', 'outflow'])

df_ab['inflow'] = inflows
df_ab['outflow'] = outflows
df_ab['rain'] = rain

save_path = save_dir / 'ablauf.csv'
df_ab.to_csv(save_path, sep=';')

### Rain analysis

In [ ]:
rain_month = ts_rain.groupby(pd.Grouper(freq='M')).sum()
print(rain_month)
rain_month.sum()